In [8]:
%pip install -q -U torch transformers bitsandbytes accelerate sentencepiece protobuf

Note: you may need to restart the kernel to use updated packages.


In [10]:
%pip install sentencepiece protobuf

Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline

MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.3"

#  4-Bit Quantization Config (Fits in ~6GB VRAM)
bnb4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

bnb8_config = BitsAndBytesConfig(
    load_in_8bit=True,
)


In [6]:
# Load Model
print(f"⏳ Loading {MODEL_NAME}...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb4_config,
    device_map="auto",
)

#  Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token # Fix for potential padding issues

print("✅ Model loaded!")

⏳ Loading mistralai/Mistral-7B-Instruct-v0.3...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

✅ Model loaded!


In [7]:
slang_term = "Rentre avec tes ieps"
prompt = f"<s>[INST] Explique cette expression d'argot français verlan: {slang_term} [/INST]"

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=150,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.1,
    do_sample=True
)

print(f"❓ Prompt: {prompt}\n")
output = pipe(prompt)
print(f"🤖 Baseline Answer:\n{output[0]['generated_text']}")

Device set to use cuda:0


❓ Prompt: <s>[INST] Explique cette expression d'argot français verlan: Rentre avec tes ieps [/INST]

🤖 Baseline Answer:
<s>[INST] Explique cette expression d'argot français verlan: Rentre avec tes ieps [/INST] L'expression "Rentre avec tes ieps" est un mot-valise en argot français, plus précisément du verlan, qui signifie "Reviens avec tes idées" (Retourne avec tes idées). Cela peut être utilisé de manière ironique ou provocatrice pour demander à quelqu'un de revenir avec des idées nouvelles, originales ou intéressantes.
